In [ ]:
import time 
import datetime
import pandas as pd
import requests
import numpy as np
import os


  <h1>
    IDŐJÁRÁS
  </h1>
  <h2>
  Bevezetés
  </h2>
  <ul>
  <li>Hány nap legyen?</li>
  <li>Milyen jellemző alapján legyen az elemzés?</li>
  <li>Milyen térképet akarok?</li>
</ul>
  <h2>
    Működés
  </h2>
    <ol>
    <li> API-kódodat be kell írni. API-kód megszerzéséért: <a href="https://home.openweathermap.org/users/sign_in">OpenWeather API</a></li>
    <li> Legfeljebb 5 napos adatot lehet "hívni". Ha többet, akkor Nan adatot fogsz kapni.</li>
    </ol>

In [ ]:
api_key="" #Ide be kell írni az API-kódodat
file =pd.read_csv("citys.csv") #Beolvasssuk a helyi koordinátait
unit = "metric" #Celsius fok
ago=5 #Hány napos adatot vigyünk be?

  <h2>
  OpenWeather API beolvasás
  </h2>

In [ ]:
def date_to_timestrap(date,delta):
        date_string = str(date)
        date = datetime.datetime.strptime(date_string, "%Y-%m-%d")  
        time_df=date-datetime.timedelta(days=delta)
        dt = str(int(datetime.datetime.timestamp(time_df)))
        return dt
    
def get(lat,lon,dt,api_key):
    json_data=requests.get("https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={dt}&units={unit}&appid={key}".format(lat=lat,lon=lon,dt=dt,unit=unit,key=api_key))
    return json_data.json()


#--------------------------------------------
data = None
count=0
index=pd.Series(np.arange(len(file)),index=file["CITY"]) #Egy adott város nevével kereshetjük a "citys" mátrix sorát
for (LAT,LON,CITY,COUNTY) in zip(file["LAT"],file["LON"],file["CITY"],file["COUNTY"]):
    for delta in range(ago):
        dt = date_to_timestrap(datetime.date.today(),delta)
        if data is None:                
            data= pd.json_normalize(get(LAT,LON,dt,api_key))
            data["county"]=COUNTY
            count+=1
        else:
            temp_temp= (pd.json_normalize(get(LAT,LON,dt,api_key)))
            data= data.append(temp_temp,ignore_index=True)
            data.loc[count,"county"]=COUNTY
            count+=1
    
data.index=pd.MultiIndex.from_product([file["CITY"],(range(1,(ago+1)))])

<h2>GeoPandas</h2>

In [ ]:
import geopandas
import pathlib
path_object = pathlib.Path("county")
path_object = pathlib.Path("county")
df = None
for a in path_object.iterdir():
    if df is None:
        df = geopandas.read_file(a)
    else:    
        df =df.append(geopandas.read_file(a),ignore_index=True)
df =df.sort_values(by=["name"])
df["centroid"]=df.centroid
df["boundary"]= df.boundary

<h2>Térkép területi, készítési függvény</h2>

In [ ]:
def terkep_terulet(df,jellemzo,cim,meddig,mettol=0):
    temp= df.copy()
    temp["jellemzo"]=data.loc[(file["CITY"],(np.arange(mettol,meddig+1))),:].groupby("county")[[jellemzo]].mean()[jellemzo].sort_index().values
    ax=temp.plot(column="jellemzo",cmap='OrRd',edgecolor='k',legend=True,figsize=(40,40))
    ax.set_title(cim,size=50)
    ax.tick_params(labelsize=40)
    fig = ax.figure
    cb_ax = fig.axes[1] 
    cb_ax.tick_params(labelsize=40)



In [ ]:
#Egy jellemzőh->X:ország megyeinek a területe színe
#Felhőség
#HŐMÉRSÉKLET

#***Emúlt  nap átlagos hőmérséklete 0--> tegnap 1, tegnap előtt 2..... 5-ig
meddig=4
mettol=4
terkep_terulet(df,"current.temp","Hőmérséklet",meddig,mettol)




<h2>Térkép ponti, készítési függvény</h2>

In [ ]:
# Egy jellemző-->X: ország megyeinek a pontjai színe
#Felhőség
def terkep_pont(df,jellemzo,cim,meddig,mettol=0):
    temp = df.copy()
    temp["jellemzo"]=data.loc[(file["CITY"],(np.arange(mettol,meddig+1))),:].groupby("county")[[jellemzo]].mean()[jellemzo].sort_index().values
    for item,para in zip(temp["centroid"],temp["jellemzo"]):
        temp["centroid"][temp["centroid"]==item] = item.buffer((0.001*para))
    ax1 = temp.plot(color="black",figsize=(40,40))
    temp = temp.set_geometry("centroid")
    ax2= temp.plot("jellemzo",ax=ax1,color="white")


In [ ]:
mettol=1
meddig=1
terkep_pont(df,"current.clouds","Felhőség",1,1)


<h2>Térkép ponti, területi készítési függvény</h2>

In [ ]:
def terkep_tp(df,file_n,szin,cim,jellemzo1,jellemzo2,meddig,mettol=0):
    temp = df.copy()
    temp["jellemzo1"]=data.loc[(file["CITY"],(np.arange(mettol,meddig+1))),:].groupby("county")[[jellemzo1]].mean()[jellemzo1].sort_index().values
    for item,para in zip(temp["centroid"],temp["jellemzo1"]):
        temp["centroid"][temp["centroid"]==item] = item.buffer((0.001*para))
    temp["jellemzo2"]=data.loc[(file["CITY"],(np.arange(mettol,meddig+1))),:].groupby("county")[[jellemzo2]].mean()[jellemzo2].sort_index().values
    ax1=temp.plot(column="jellemzo2",cmap=szin,edgecolor='k',legend=True,figsize=(40,40))
    ax1.set_title("{}-{} közti nap:{} (Kör-Terület)".format(mettol,meddig,cim),size=50)
    ax1.tick_params(labelsize=40)
    fig = ax1.figure
    cb_ax = fig.axes[1] 
    cb_ax.tick_params(labelsize=40)
    temp = temp.set_geometry("centroid")
    ax2= temp.plot("jellemzo1",ax=ax1,color="white")
    filepath = os.path.join("maps/", '{}{}_map_{}.jpg'.format(mettol,meddig,file_n))
    chart = ax2.get_figure()
    chart.savefig(filepath, dpi=300)

In [ ]:
mettol=1
meddig=1
jellemzo1="current.clouds"
jellemzo2="current.temp"
cim="Felhő-hőmérséklet viszony"
szin='OrRd'
file_n="c_t"
for mettol, meddig in zip(range(1,6),range(1,6)):  
    terkep_tp(df,file_n,szin,cim,jellemzo1,jellemzo2,meddig,mettol)

In [ ]:
mettol=1
meddig=1
jellemzo1="current.clouds"
jellemzo2="current.wind_speed"
cim="Szelesség-Felhőség viszony"
szin='Blues'
file_n="c_ws"
for mettol, meddig in zip(range(1,6),range(1,6)):  
    terkep_tp(df,file_n,szin,cim,jellemzo1,jellemzo2,meddig,mettol)
    

In [ ]:
data